# ***Social Network Analysis***

In [1]:
pip install dash dash-cytoscape networkx python-louvain pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 37.1 MB/s eta 0:00:00
  Created wheel for dash-cytoscape: filename=dash_cytoscape-1.0.2-py3-none-any.whl size=4010717 sha256=08b7fb189b9ca0af82ed29ee6287b732e22357a164e1ab2379daa8d9fb9e0865
  Stored in directory: /root/.cache/pip/wheels/91/23/5e/56fa701c668444b121ad2353a96478179dc49086a9c44ee930
Successfully built dash-cytoscape


In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_cytoscape as cyto
import networkx as nx
import community.community_louvain as community_louvain
import pandas as pd

# Dataset 1: Facebook Circles Dataset
url_facebook = "https://snap.stanford.edu/data/facebook_combined.txt.gz"
df_facebook = pd.read_csv(url_facebook, delimiter=' ', header=None, names=['source', 'target'])
G_facebook = nx.from_pandas_edgelist(df_facebook, source='source', target='target')
partition_facebook = community_louvain.best_partition(G_facebook)
for node, community in partition_facebook.items():
    G_facebook.nodes[node]['community'] = community

# Dataset 2: Karate Club Dataset
G_karate = nx.karate_club_graph()
partition_karate = community_louvain.best_partition(G_karate)
for node, community in partition_karate.items():
    G_karate.nodes[node]['community'] = community


In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_cytoscape as cyto
import networkx as nx
import community.community_louvain as community_louvain
import pandas as pd


In [4]:
# Dataset 1: Facebook Circles Dataset
url_facebook = "https://snap.stanford.edu/data/facebook_combined.txt.gz"
df_facebook = pd.read_csv(url_facebook, delimiter=' ', header=None, names=['source', 'target'])
G_facebook = nx.from_pandas_edgelist(df_facebook, source='source', target='target')
partition_facebook = community_louvain.best_partition(G_facebook)
for node, community in partition_facebook.items():
    G_facebook.nodes[node]['community'] = community

# Dataset 2: Karate Club Dataset
G_karate = nx.karate_club_graph()
partition_karate = community_louvain.best_partition(G_karate)
for node, community in partition_karate.items():
    G_karate.nodes[node]['community'] = community


In [5]:
def generate_cytoscape_data(graph, partition, num_nodes):
    subgraph = graph.subgraph(list(graph.nodes)[:num_nodes])

    elements = []
    for node in subgraph.nodes:
        elements.append({
            'data': {'id': str(node), 'label': str(node), 'community': partition[node]},
            'classes': 'community-' + str(partition[node])
        })
    for edge in subgraph.edges:
        elements.append({
            'data': {'source': str(edge[0]), 'target': str(edge[1])}
        })
    return elements


In [6]:
# Dash App and Layout
app = dash.Dash(__name__)

app.layout = html.Div(
    style={'backgroundColor': '#111111', 'color': '#FFFFFF', 'padding': '10px', 'height': '100vh', 'display': 'flex', 'flexDirection': 'column'},
    children=[
        # Header
        html.H1('Social Network Analysis Dashboard', style={'text-align': 'center', 'color': '#00FFAA'}),

        # Dropdown to select dataset
        html.Div([
            html.Label('Select Dataset:', style={'color': '#FFFFFF'}),
            dcc.Dropdown(
                id='dataset-dropdown',
                options=[
                    {'label': 'Facebook Circles', 'value': 'facebook'},
                    {'label': 'Karate Club', 'value': 'karate'}
                ],
                value='facebook',
                style={'color': '#000000'}
            )
        ], style={'width': '300px', 'margin': '0 auto 20px'}),

        # Slider to control number of nodes displayed
        html.Div([
            html.Label('Number of Nodes:', style={'color': '#FFFFFF'}),
            dcc.Slider(
                id='node-slider',
                min=10, max=500, step=10, value=100,
                marks={i: str(i) for i in range(0, 501, 100)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={'margin': '20px 0'}),

        # Network graph with transparent background
        cyto.Cytoscape(
            id='network-graph',
            layout={'name': 'cose'},  # Layout style for graph
            style={'width': '100%', 'height': '100%', 'backgroundColor': 'transparent', 'flexGrow': '1'},
            elements=generate_cytoscape_data(G_facebook, partition_facebook, 100),
            stylesheet=[
                {
                    'selector': 'node',
                    'style': {
                        'background-color': '#00FFAA',
                        'label': 'data(label)',
                        'color': 'white',
                        'text-outline-width': '1px',
                        'text-outline-color': '#000000'
                    }
                },
                {
                    'selector': 'edge',
                    'style': {
                        'line-color': '#888',
                        'width': 1
                    }
                },
                {
                    'selector': '.community-0',
                    'style': {'background-color': '#FF4136'}
                },
                {
                    'selector': '.community-1',
                    'style': {'background-color': '#FF851B'}
                },
                {
                    'selector': '.community-2',
                    'style': {'background-color': '#0074D9'}
                },
                {
                    'selector': '.community-3',
                    'style': {'background-color': '#2ECC40'}
                }
            ]
        ),

        # Footer
        html.Div('Created by AI Innovators', style={'text-align': 'center', 'marginTop': '20px', 'color': '#AAAAAA'})
    ]
)


In [7]:
# Callback to update the graph based on the dataset and slider input
@app.callback(
    Output('network-graph', 'elements'),
    [Input('dataset-dropdown', 'value'),
     Input('node-slider', 'value')]
)
def update_graph(selected_dataset, num_nodes):
    if selected_dataset == 'facebook':
        return generate_cytoscape_data(G_facebook, partition_facebook, num_nodes)
    elif selected_dataset == 'karate':
        return generate_cytoscape_data(G_karate, partition_karate, num_nodes)


In [8]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=False)


<IPython.core.display.Javascript object>

# **HAPPY COADING**